In [ ]:
from faker import Faker
fake = Faker('pt_BR')
import datetime, json, requests
from dateutil.relativedelta import relativedelta
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey


HOST = 'https://padariacasedb.documents.azure.com:443/'
MASTER_KEY = 'KPxz6QHRsC0rnSC0A2Wi4C2PNVapVy0JU9GpCyUC4LRrD9Xqrmh18BCShyvXTHOrUxyNIFEf848tHGmv221eGA=='
DATABASE_ID = 'padariadb'
CONTAINER_ID = 'providers'

In [ ]:
client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY} )
db = client.create_database_if_not_exists(id=DATABASE_ID)
container = db.create_container_if_not_exists(
    id=CONTAINER_ID, 
    partition_key=PartitionKey(path="/providerId"),
    offer_throughput=400
)

In [ ]:
# enable_cross_partition_query should be set to True as the container is partitioned
items = list(container.query_items(
    query="SELECT top 1 c.id as customerId,c.originBranchId as branchId,c.cpf,c.name FROM customer c where RAND() <= @id",
    parameters=[
        { "name":"@id", "value": .3 }
    ],
    enable_cross_partition_query=True
))


In [ ]:
for x in range(5000):
    date = fake.date_between(start_date='-30y', end_date='today')
    Provider = {
    "id": fake.hexify(text='^^^^-^^^^-^^^^-^^^^'),
    "providerId": x+1,
    "originBranchId": fake.random_number(3),
    "companyName": fake.company(),
    "cnpj": fake.cnpj(),
    "createdDate": date.strftime('%Y-%m-%d'),
    "email": fake.email(),
    "phone": fake.phone_number(),
    "enabled": True,
    "addresses": [{
        "addressStreet": fake.street_name(),
        "addressNumber": fake.building_number(),
        "addressCity": fake.city(),
        "addressState": fake.state_abbr(),
        "addressCountry": fake.current_country(),
        "addressPostalCode": fake.postcode(),
        "addressComplement": fake.neighborhood(),
        "isMain": True
    }],
    "summary":{
        "totalProducts": 0,
        "lastSupply": "null"
    }
    }
    print (x)
    container.create_item(body=Provider)

In [ ]:
#deletar todos fornecedores
item_list = list(container.read_all_items(max_item_count=20))

print('Found {0} items'.format(item_list.__len__()))

for doc in item_list:
    print(doc.get('providerId'))
    container.delete_item(item=doc.get('id'), partition_key=doc.get('providerId'))